# Imports

In [365]:
import os

import pandas as pd
import numpy as np

import json

from ollama import chat
from ollama import ChatResponse
from ollama import ps

from pydantic import BaseModel

import re

# Dados

In [161]:
# df com os processos e dados de capa
path = os.path.join('..','dados','Embasa_sem_duplicidade.xlsx')
df_processos = pd.read_excel(path, index_col='processo')

In [162]:
df_processos

,cpf_cnpj,data_inicio_processo,data_ultima_movimentação,data_ultima_verificação,fontes_tribunais_estao_arquivadas,dados_capa_UI,dados_capa_agente
processo,,,,,,,
0002153-53.2010.5.05.0531,13.504.675/0001-10,0010-10-04,2023-09-22,2024-09-29,Não,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal Regional do Trabalho da 5ª R...
0000082-80.1983.8.05.0141,13.504.675/0001-10,1983-07-25,2014-07-14,2023-12-31,Não,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...
0008104-82.2010.8.05.0141,13.504.675/0001-10,1983-07-25,2010-11-23,2024-06-13,Sim,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...
0000082-98.1985.8.05.0274,13.504.675/0001-10,1985-10-28,2022-04-13,2024-01-11,Não,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...
0009540-51.1986.8.05.0001,13.504.675/0001-10,1986-05-08,2021-10-28,2024-01-11,Não,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...
...,...,...,...,...,...,...,...
0004034-83.2025.8.05.0080,13.504.675/0001-10,2025-02-07,2025-02-07,2025-02-07,Não,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...
0000168-86.2025.8.05.0106,13.504.675/0001-10,2025-02-07,2025-02-07,2025-02-07,Não,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...
0023080-04.2025.8.05.0001,13.504.675/0001-10,2025-02-07,2025-02-07,2025-02-07,Não,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...


In [165]:
path = os.path.join('..','dados','EMBASA_v3.xlsx')
sheet_name = 'Total Relevantes - processos'
df_relevantes = pd.read_excel(path, sheet_name=sheet_name, index_col='processo')

In [166]:
df_relevantes

,cpf_cnpj,id,processo_fonte_id,sigla,descricao,nome,grau,polo,polo_tipo_normalizado,polo_ativo,...,status_predito,situacao,url,informacoes_complementares,classe,area,categoria,assunto,assunto_principal_normalizado,assuntos_normalizados
processo,,,,,,,,,,,,,,,,,,,,,
0119105-41.2009.8.05.0001,13.504.675/0001-10,1849,7756091,TJBA,TJBA - 1º grau,Tribunal de Justiça da Bahia,1,PASSIVO,Réu,Rosimeire Ferreira dos Santos Cerqueira,...,ATIVO,não disponível,https://consultapublicapje.tjba.jus.br/pje/Con...,NaN,AÇÃO POPULAR (66),não disponível,DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...,Dano ao Erário,DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...,DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...
0000165-87.2011.8.05.0053,13.504.675/0001-10,1849,665582294,TJBA,TJBA - 1º grau,Tribunal de Justiça da Bahia,1,PASSIVO,Réu,não disponível,...,ATIVO,não disponível,https://consultapublicapje.tjba.jus.br/pje/Con...,Jurisdição : CASTRO ALVES,AÇÃO CIVIL PÚBLICA (65),não disponível,DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...,Dano ao Erário,DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...,DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...
0005159-78.2011.8.05.0112,13.504.675/0001-10,18064,660834040,STF,STF - Superior,Supremo Tribunal Federal,3,ATIVO,Recorrente,Elves Ferreira Almeida,...,ATIVO,não disponível,https://portal.stf.jus.br:443/processos/detalh...,NaN,não disponível,não disponível,DIREITO PROCESSUAL CIVIL E DO TRABALHO,DIREITO PROCESSUAL CIVIL E DO TRABALHO | Proce...,DIREITO PROCESSUAL CIVIL E DO TRABALHO > Proce...,não disponível
0011421-50.2015.8.05.0000,13.504.675/0001-10,183,621052065,TJBA,TJBA - 2º grau,Tribunal de Justiça da Bahia,2,PASSIVO,Réu,Empresa Baiana de Aguas e Saneamento S.A,...,INATIVO,Baixado,http://esaj.tjba.jus.br/cpo/sg/search.do?conve...,Processo : 0011421-50.2015.8.05.0000 BaixadoOr...,Ação Popular,Cível,DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...,Dano ao Erário,DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...,não disponível
0509414-88.2016.8.05.0001,13.504.675/0001-10,5911,604315287,TJBA,TJBA - 2º grau,Tribunal de Justiça da Bahia,2,PASSIVO,Embargado,Vanderleia Nascimento da Silva,...,INATIVO,não disponível,https://pje2g.tjba.jus.br/pje/ConsultaPublica/...,Órgão Julgador Colegiado : Quarta Câmara Cível...,EMBARGOS DE DECLARAÇÃO CÍVEL (1689),não disponível,DIREITO DO CONSUMIDOR,Fornecimento de Água,DIREITO DO CONSUMIDOR > Contratos de Consumo >...,DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0000234-98.2024.5.05.0026,13.504.675/0001-10,35,670112791,TRT-5,TRT-5 - 1º grau,Tribunal Regional do Trabalho da 5ª Região,1,PASSIVO,Réu,Fabio dos Santos Pinho,...,ATIVO,não disponível,NaN,Justiça Gratuita : SimDigital : False,ATOrd,Trabalhista,DIREITO DO TRABALHO,Doença Ocupacional,DIREITO DO TRABALHO > Direito Individual do Tr...,DIREITO DO TRABALHO > Direito Individual do Tr...
0000454-74.2024.5.05.0001,13.504.675/0001-10,35,702757830,TRT-5,TRT-5 - 1º grau,Tribunal Regional do Trabalho da 5ª Região,1,PASSIVO,Réu,Nivaldo Soares de Brito,...,ATIVO,não disponível,NaN,NaN,Ação Trabalhista - Rito Sumaríssimo,Trabalhista,DIREITO DO TRABALHO,Dispensa Discriminatória,DIREITO DO TRABALHO > Direito Individual do Tr...,DIREITO DO TRABALHO > Direito Individual do Tr...
0000700-19.2024.5.05.0018,13.504.675/0001-10,35,732694232,TRT-5,TRT-5 - 1º grau,Tribunal Regional do Trabalho da 5ª Região,1,PASSIVO,Réu,Cléber Castro dos Santos,...,ATIVO,não disponível,NaN,NaN,Ação Trabalhista - Rito Sumaríssimo,Trabalhista,DIREITO DO TRABALHO,Adicional de Hora Extra,DIREITO DO TRABALHO > Direito Individual do Tr...,DIREITO DO TRABALHO > Direito Individual do Tr...


In [169]:
df_gabarito = df_processos.copy()
df_gabarito['classificacao'] = np.where(df_processos.index.isin(df_relevantes.index), 'S', 'N')

In [172]:
df_gabarito['classificacao'].value_counts()

classificacao
N    109413
S       604
Name: count, dtype: int64

## Amostra

In [379]:
sample_size = 3

In [380]:
df_amostra_relevantes = df_gabarito.loc[df_gabarito['classificacao'] == 'S'].sample(sample_size, random_state=0)
df_amostra_relevantes

,cpf_cnpj,data_inicio_processo,data_ultima_movimentação,data_ultima_verificação,fontes_tribunais_estao_arquivadas,dados_capa_UI,dados_capa_agente,classificacao
processo,,,,,,,,
0000148-21.2024.5.05.0611,13.504.675/0001-10,2024-02-17,2025-01-29,2025-01-29,Não,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal Regional do Trabalho da 5ª R...,S
0144162-37.2004.8.05.0001,13.504.675/0001-10,2009-01-27,2018-10-16,2024-04-08,Sim,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...,S
8000258-50.2023.8.05.0001,13.504.675/0001-10,2023-01-06,2024-02-24,2024-06-22,Não,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...,S


In [381]:
df_amostra_nao_relevantes = df_gabarito.loc[df_gabarito['classificacao'] == 'N'].sample(sample_size, random_state=0)
df_amostra_nao_relevantes

,cpf_cnpj,data_inicio_processo,data_ultima_movimentação,data_ultima_verificação,fontes_tribunais_estao_arquivadas,dados_capa_UI,dados_capa_agente,classificacao
processo,,,,,,,,
0062102-16.2018.8.05.0001,13.504.675/0001-10,2018-05-14,2019-11-28,2024-10-26,Sim,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...,N
0011947-18.2023.8.05.0103,13.504.675/0001-10,2023-10-13,2024-02-20,2024-02-23,Não,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...,N
0163407-72.2020.8.05.0001,13.504.675/0001-10,2020-11-30,2022-11-02,2024-10-25,Sim,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...,N


In [382]:
df_amostra = pd.concat([df_amostra_relevantes,df_amostra_nao_relevantes])
df_amostra

,cpf_cnpj,data_inicio_processo,data_ultima_movimentação,data_ultima_verificação,fontes_tribunais_estao_arquivadas,dados_capa_UI,dados_capa_agente,classificacao
processo,,,,,,,,
0000148-21.2024.5.05.0611,13.504.675/0001-10,2024-02-17,2025-01-29,2025-01-29,Não,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal Regional do Trabalho da 5ª R...,S
0144162-37.2004.8.05.0001,13.504.675/0001-10,2009-01-27,2018-10-16,2024-04-08,Sim,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...,S
8000258-50.2023.8.05.0001,13.504.675/0001-10,2023-01-06,2024-02-24,2024-06-22,Não,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...,S
0062102-16.2018.8.05.0001,13.504.675/0001-10,2018-05-14,2019-11-28,2024-10-26,Sim,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...,N
0011947-18.2023.8.05.0103,13.504.675/0001-10,2023-10-13,2024-02-20,2024-02-23,Não,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...,N
0163407-72.2020.8.05.0001,13.504.675/0001-10,2020-11-30,2022-11-02,2024-10-25,Sim,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...,N


In [290]:
# # Processo Relevante
# dados_de_capa = df_processos.loc['0001267-83.1999.8.05.0274','dados_capa_agente']
# dados_de_capa

In [127]:
# # Processo Relevante
# dados_de_capa = df_processos.loc['0002153-53.2010.5.05.0531','dados_capa_agente']
# dados_de_capa

'<<Fonte: Tribunal Regional do Trabalho da 5ª Região\nClasse da Ação: Ação Trabalhista - Rito Sumaríssimo\nÁrea do Direito: DIREITO DO TRABALHO\nAssunto(s): Contrato Individual de Trabalho, FGTS, Duração do Trabalho, Rescisão do Contrato de Trabalho, Verbas Rescisórias, Aviso Prévio, Décimo Terceiro Salário Proporcional, Férias Proporcionais, Multa de 40% do FGTS, Multa do Artigo 467 da CLT , Multa do Artigo  477 da CLT, Saldo de Salário\n>>\n\n<<Fonte: Tribunal Regional do Trabalho da 5ª Região\nClasse da Ação: Recurso Ordinário Trabalhista\nÁrea do Direito: Trabalhista\nAssunto(s): não disponível\n>>\n\n'

# Parâmetros LLM

In [221]:
# inputs
usuario = 'andcm'

In [116]:
# prompt = """
# - Dados do processo: Ação Trabalhista, Dispensa Discriminatória. 
# - Pergunta: os dados do processo trazem o termo 'discriminação' ou sinônimos jurídicos? 
# - Flag "S": o processo envolve discriminação.
# - Flag "N": o processo não envolve discriminação
#  Responda **exatamente** no seguinte formato JSON, sem qualquer outro texto, explicações ou formatação extra:
# ```json
# {
#   "flag": "S", // ou "N"
#   "motivo": "descrição do motivo"
# }
# """

In [297]:
# Pydantic
class ClassificacaoProcessos(BaseModel):
  flag: str
  motivo: str

In [223]:
# # JSON Schema
# json_schema = """
# ```json
# {
#     "flag": "S", // ou "N", // ou "ND"
#     "motivo": "descrição do motivo"
# }
# """

In [298]:
# JSON Schema
json_schema = """
```json
{
    "flag": "S", // ou "N".
    "motivo": "descrição do motivo da flag adotada."
}
"""

In [ ]:
# def make_prompt(processo, dados_de_capa, json_schema, usuario):
#     prompt = f"""
# - Auxilie o usuário {usuario} a identificar se o processo {processo} é: i) **relevante**, ii) **não relevante**, ou iii) **sem dados disponíveis**.

# Utilizar somente as **informações do processo** a seguir:
# {dados_de_capa}

# ---

# Classificar o processo utilizando os seguintes critérios:
# i) O processo é RELEVANTE caso algum dos temas listados abaixo estejam presentes nas **informações do processo**:
# - Ação Civil de Improbidade Administrativa
# - Ação Civil Pública
# - Ação Popular
# - Ação Coletiva
# - Ação Penal, Ação Criminal
# - crime, penal, carta precatória criminal, habeas corpus
# - crimes contra o meio ambiente
# - assédio moral ou sexual
# - trabalho infantil
# - trabalho escravo ou trabalho em condições análogas à escravidão
# - violência contra a mulher, idoso ou pessoa com deficiência
# - crime resultante de discriminação racial e de etnia
# - discriminação, dispensa discriminatória
# - lavagem de dinheiro ou ocultação de bens, direitos e valores
# - exploração sexual ou proveito criminoso da prostituição
# - tráfico de pessoas
# - danos contra a administração pública
# - danos contra a ordem econômica e tributária
# - danos contra o sistema financeiro
# - danos contra o mercado de capitais

# ii) O processo é **não relevante** caso não tenha nenhum tema de relevância nas **informações do processo**.

# iii) O processo é **sem dados disponíveis** quando não há NENHUM dado disponível nas informações do processo.

# ---

# A saída deve ter estritamente os seguintes itens:
# - Uma flag ESTRITAMENTE no formato 'S', 'N', ou 'ND':
#   - 'S' quando o processo for RELEVANTE
#   - 'N' quando o processo for NÃO RELEVANTE
#   - 'ND' quando não há NENHUM dado disponível nas informações do processo
# - Motivo: o motivo da flag adotada, informando, se possível, do que se trata o processo

# ---
# Responda **exatamente** no seguinte formato JSON, sem qualquer outro texto, explicações ou formatação extra:
# {json_schema}

# """
#     return prompt

In [280]:
# def make_prompt(processo, dados_de_capa, json_schema, usuario):
#     prompt = f"""
# Utilize as **informações do processo {processo}** abaixo:
# {dados_de_capa}

# ---

# 1. Classifique o processo como **relevante** se algum dos temas abaixo estiver presente nas **informações do processo**
# - Ação Civil de Improbidade Administrativa
# - Ação Civil Pública
# - Ação Popular
# - Ação Coletiva
# - Ação Penal, Ação Criminal
# - crime, penal, carta precatória criminal, habeas corpus
# - crimes contra o meio ambiente
# - assédio moral ou sexual
# - trabalho infantil
# - trabalho escravo ou trabalho em condições análogas à escravidão
# - violência contra a mulher, idoso ou pessoa com deficiência
# - crime resultante de discriminação racial e de etnia
# - discriminação, dispensa discriminatória
# - lavagem de dinheiro ou ocultação de bens, direitos e valores
# - exploração sexual ou proveito criminoso da prostituição
# - tráfico de pessoas
# - danos contra a administração pública
# - danos contra a ordem econômica e tributária
# - danos contra o sistema financeiro
# - danos contra o mercado de capitais

# 2. Do contrário, o processo é **não relevante**

# 3. Se não houver nenhum dado disponível nas **informações do processo**, o processo é **sem dados disponíveis**

# ---

# Formato da resposta:
# - Flag "S": o processo é **relevante**.
# - Flag "N": o processo é **não relevante**.
# - Flag "ND": o processo é sem **dados disponíveis**.
#  Responda **exatamente** no seguinte formato JSON, sem qualquer outro texto, explicações ou formatação extra:
# {json_schema}

# """
#     return prompt

In [317]:
# def make_prompt(processo, dados_de_capa, json_schema, usuario):
#     prompt = f"""
# Utilize as **informações do processo {processo}**:
# {dados_de_capa}

# ---

# 1. O processo é **relevante** caso as **informações do processo** contenham pelo menos um dos temas listados abaixo, ou termos jurídicos semelhantes:
# - Ação Civil de Improbidade Administrativa
# - Ação Civil Pública
# - Ação Popular
# - Ação Coletiva
# - Ação Penal, Ação Criminal
# - crime, penal, carta precatória criminal, habeas corpus
# - crimes contra o meio ambiente
# - assédio moral ou sexual
# - trabalho infantil
# - trabalho escravo ou trabalho em condições análogas à escravidão
# - violência contra a mulher, idoso ou pessoa com deficiência
# - crime resultante de discriminação racial e de etnia
# - discriminação, dispensa discriminatória
# - lavagem de dinheiro ou ocultação de bens, direitos e valores
# - exploração sexual ou proveito criminoso da prostituição
# - tráfico de pessoas
# - danos contra a administração pública
# - danos contra a ordem econômica e tributária
# - danos contra o sistema financeiro
# - danos contra o mercado de capitais

# 2. O processo é **não relevante** caso as informações do processo não contenham nenhum dos temas indicados como relevantes.

# ---

# Formato da resposta:
# - Flag "S": o processo é **relevante**.
# - Flag "N": o processo é **não relevante**.
#  Responda **exatamente** no seguinte formato JSON, sem qualquer outro texto, explicações ou formatação extra:
# {json_schema}

# """
#     return prompt

In [394]:
def make_prompt(processo, dados_de_capa, json_schema, usuario):
    prompt = f"""
Responda e raciocine em português

---

Utilize as informações do processo {processo}:
{dados_de_capa}

Utilize somente os campos "Classe da Ação" e "Assunto(s)"
Não raciocine, só busque as palavras chave nos campos indicados.

---

1. O processo é **relevante** caso a **classe da ação** // ou **assunto(s)** contenham **estritamente** alguma das palavras chaves listadas abaixo:
- Ação Civil de Improbidade Administrativa
- Ação Civil Pública
- Ação Popular
- Ação Coletiva
- Ação Penal, Ação Criminal
- crime, penal, carta precatória criminal, habeas corpus
- crimes contra o meio ambiente
- assédio moral ou sexual
- trabalho infantil
- trabalho escravo ou trabalho em condições análogas à escravidão
- violência contra a mulher, idoso ou pessoa com deficiência
- crime resultante de discriminação racial e de etnia
- discriminação, dispensa discriminatória
- lavagem de dinheiro ou ocultação de bens, direitos e valores
- exploração sexual ou proveito criminoso da prostituição
- tráfico de pessoas
- danos contra a administração pública
- danos contra a ordem econômica e tributária
- danos contra o sistema financeiro
- danos contra o mercado de capitais

2. O processo é **não relevante** caso a **classe da ação** // ou **assunto(s)** não contiverem nenhuma das palavras chave indicadas listadas como relevantes.

---

Formato da resposta:
- Flag "S": o processo é **relevante**.
- Flag "N": o processo é **não relevante**.
 Responda ao final **exatamente** no seguinte formato JSON, sem qualquer outro texto, explicações ou formatação extra:
{json_schema}

"""
    return prompt

In [333]:
# # usuario = usuario
# # processo = processo
# # dados_de_capa = dados_de_capa

# prompt = f"""
# - Auxilie o usuário {usuario} a identificar se o processo {processo} é: i) RELEVANTE, ii) NãO RELEVANTE, ou iii) SEM DADOS DISPONÍVEIS: não há dados disponíveis para a classificação.

# Utilizar somente as informações do processo a seguir:
# {dados_de_capa}

# ---

# Classificar o processo utilizando os seguintes critérios:
# i) O processo é RELEVANTE caso as informações do processo contenham PELO MENOS UM dos temas listados abaixo, ou termos jurídicos semelhantes:
# - Ação Civil de Improbidade Administrativa
# - Ação Civil Pública
# - Ação Popular
# - Ação Coletiva
# - Ação Penal, Ação Criminal
# - crime, penal, carta precatória criminal, habeas corpus
# - crimes contra o meio ambiente
# - assédio moral ou sexual
# - trabalho infantil
# - trabalho escravo ou trabalho em condições análogas à escravidão
# - violência contra a mulher, idoso ou pessoa com deficiência
# - crime resultante de discriminação racial e de etnia
# - discriminação, dispensa discriminatória
# - lavagem de dinheiro ou ocultação de bens, direitos e valores
# - exploração sexual ou proveito criminoso da prostituição
# - tráfico de pessoas
# - danos contra a administração pública
# - danos contra a ordem econômica e tributária
# - danos contra o sistema financeiro
# - danos contra o mercado de capitais

# ii) O processo é NÂO RELEVANTE caso as informações do processo não contenham NENHUM dos temas indicados como RELEVANTES.

# iii) O processo é SEM DADOS DISPONÍVEIS quando não há NENHUM dado disponível nas informações do processo.

# ---

# A saída deve ter estritamente os seguintes itens:
# - Uma flag ESTRITAMENTE no formato 'S', 'N', ou 'ND':
#   - 'S' quando o processo for RELEVANTE
#   - 'N' quando o processo for NÃO RELEVANTE
#   - 'ND' quando não há NENHUM dado disponível nas informações do processo
# - Motivo: o motivo da flag adotada, informando, se possível, do que se trata o processo

# ---

# {json_schema}

# """

# Teste Processo Individual

In [334]:
# prompt = """
# - Dados do processo: Ação Trabalhista, Dispensa Discriminatória. 
# - Pergunta: os dados do processo trazem o termo 'discriminação' ou sinônimos jurídicos? 
# - Flag "S": o processo envolve discriminação.
# - Flag "N": o processo não envolve discriminação
#  Responda **exatamente** no seguinte formato JSON, sem qualquer outro texto, explicações ou formatação extra:
# ```json
# {
#   "flag": "S", // ou "N"
#   "motivo": "descrição do motivo"
# }
# """

In [335]:
# prompt = """
# - Dados do processo: Ação Trabalhista, Dispensa Discriminatória. 
# - Pergunta: os dados do processo trazem o termo 'discriminação' ou sinônimos jurídicos? 
# - Flag "S": o processo envolve discriminação.
# - Flag "N": o processo não envolve discriminação
# """

In [357]:
processo = '0001267-83.1999.8.05.0274'
dados_de_capa = df_processos.loc[processo,'dados_capa_agente']

In [358]:
prompt = make_prompt(processo, dados_de_capa, json_schema, usuario)

In [346]:
response: ChatResponse = chat(model='deepseek-r1:1.5b', 
                              messages = [{'role': 'user','content':prompt}],
                              format = ClassificacaoProcessos.model_json_schema(),
                              options = {'temperature': 0}     
                              )

In [347]:
tokens_input = response.prompt_eval_count
tokens_output = response.eval_count
segundos = response.total_duration*1e-9
output_estruturado = ClassificacaoProcessos.model_validate_json(response.message.content)

print(f"""
Prompt: {prompt}

Output: {response.message.content}

Output_estruturado: {output_estruturado} 

----------------------------------------------
input_tokens: {tokens_input}
output_tokens: {tokens_output}

tempo: {segundos:.2f} segundos
tokens/segundo: {(tokens_input + tokens_output)/ segundos:.2f}

""")


Prompt: 
Utilize as informações do processo 0001267-83.1999.8.05.0274:
<<Fonte: Tribunal de Justiça da Bahia
Classe da Ação: ACAO CIVIL PUBLICA
Área do Direito: não disponível
Assunto(s): não disponível
>>

<<Fonte: Tribunal de Justiça da Bahia
Classe da Ação: Ação Civil Pública
Área do Direito: Cível
Assunto(s): não disponível
>>



---

1. O processo é **relevante** caso a **classe da ação** // ou **assunto** contenham alguns dos termos listados abaixo:
- Ação Civil de Improbidade Administrativa
- Ação Civil Pública
- Ação Popular
- Ação Coletiva
- Ação Penal, Ação Criminal
- crime, penal, carta precatória criminal, habeas corpus
- crimes contra o meio ambiente
- assédio moral ou sexual
- trabalho infantil
- trabalho escravo ou trabalho em condições análogas à escravidão
- violência contra a mulher, idoso ou pessoa com deficiência
- crime resultante de discriminação racial e de etnia
- discriminação, dispensa discriminatória
- lavagem de dinheiro ou ocultação de bens, direitos e val

In [359]:
response: ChatResponse = chat(model='deepseek-r1:1.5b', 
                              messages = [{'role': 'user','content':prompt}],
                              options = {'temperature': 0}     
                              )

In [367]:
tokens_input = response.prompt_eval_count
tokens_output = response.eval_count
segundos = response.total_duration*1e-9
# output_estruturado
regex = r'(\{\s*"flag":\s*".*?",\s*"motivo":\s*".*?"\s*\})'
match = re.search(regex, response.message.content)
if match:
    output_estruturado = match.group()
    output_estruturado = json.loads(output_estruturado)

print(f"""
Prompt: {prompt}

Output: {response.message.content}

Output_estruturado: {output_estruturado} 

----------------------------------------------
input_tokens: {tokens_input}
output_tokens: {tokens_output}

tempo: {segundos:.2f} segundos
tokens/segundo: {(tokens_input + tokens_output)/ segundos:.2f}

""")


Prompt: 
Utilize as informações do processo 0001267-83.1999.8.05.0274:
<<Fonte: Tribunal de Justiça da Bahia
Classe da Ação: ACAO CIVIL PUBLICA
Área do Direito: não disponível
Assunto(s): não disponível
>>

<<Fonte: Tribunal de Justiça da Bahia
Classe da Ação: Ação Civil Pública
Área do Direito: Cível
Assunto(s): não disponível
>>



---

1. O processo é **relevante** caso a **classe da ação** // ou **assunto** contenham alguns dos termos listados abaixo:
- Ação Civil de Improbidade Administrativa
- Ação Civil Pública
- Ação Popular
- Ação Coletiva
- Ação Penal, Ação Criminal
- crime, penal, carta precatória criminal, habeas corpus
- crimes contra o meio ambiente
- assédio moral ou sexual
- trabalho infantil
- trabalho escravo ou trabalho em condições análogas à escravidão
- violência contra a mulher, idoso ou pessoa com deficiência
- crime resultante de discriminação racial e de etnia
- discriminação, dispensa discriminatória
- lavagem de dinheiro ou ocultação de bens, direitos e val

In [361]:
output_estruturado

'{'

In [311]:
for a in dir(response):
    if a[:1] != '_':
        print(f"{a}: {getattr(response,a)}")

construct: <bound method BaseModel.construct of <class 'ollama._types.ChatResponse'>>
copy: <bound method BaseModel.copy of ChatResponse(model='deepseek-r1:1.5b', created_at='2025-02-10T01:12:34.480034548Z', done=True, done_reason='stop', total_duration=6815994318, load_duration=17175934, prompt_eval_count=485, prompt_eval_duration=1224000000, eval_count=69, eval_duration=5573000000, message=Message(role='assistant', content='{"flag": "N", "motivo": "O processo não está incluído nas informações fornecidas. O mainé e o tráfico de pessoas são processos que podem ser relevantes, mas não estão presentes no texto"}\n    \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t', images=None, tool_calls=None))>
created_at: 2025-02-10T01:12:34.480034548Z
dict: <bound method BaseModel.dict of ChatResponse(model='deepseek-r1:1.5b', created_at='2025-02-10T01:12:34.480034548Z', done=True, done_reason='stop', total_duration=6815994318, load_duration=17175934, prompt_eval_count=485, prompt_eval_duration=1224000000, eval_co

In [155]:
ps()

ProcessResponse(models=[Model(model='deepseek-r1:1.5b', name='deepseek-r1:1.5b', digest='a42b25d8c10a841bd24724309898ae851466696a7d7f3a0a408b895538ccbc96', expires_at=datetime.datetime(2025, 2, 9, 15, 55, 38, 26085, tzinfo=TzInfo(-03:00)), size=1567668736, size_vram=0, details=ModelDetails(parent_model='', format='gguf', family='qwen2', families=['qwen2'], parameter_size='1.8B', quantization_level='Q4_K_M'))])

# Teste Amostra

In [390]:
df_amostra

,cpf_cnpj,data_inicio_processo,data_ultima_movimentação,data_ultima_verificação,fontes_tribunais_estao_arquivadas,dados_capa_UI,dados_capa_agente,classificacao
processo,,,,,,,,
0000148-21.2024.5.05.0611,13.504.675/0001-10,2024-02-17,2025-01-29,2025-01-29,Não,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal Regional do Trabalho da 5ª R...,S
0144162-37.2004.8.05.0001,13.504.675/0001-10,2009-01-27,2018-10-16,2024-04-08,Sim,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...,S
8000258-50.2023.8.05.0001,13.504.675/0001-10,2023-01-06,2024-02-24,2024-06-22,Não,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...,S
0062102-16.2018.8.05.0001,13.504.675/0001-10,2018-05-14,2019-11-28,2024-10-26,Sim,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...,N
0011947-18.2023.8.05.0103,13.504.675/0001-10,2023-10-13,2024-02-20,2024-02-23,Não,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...,N
0163407-72.2020.8.05.0001,13.504.675/0001-10,2020-11-30,2022-11-02,2024-10-25,Sim,**Dados de Capa**:\n\n**Processos Relacionados...,<<Fonte: Tribunal de Justiça da Bahia\nClasse ...,N


In [391]:
# dict_results = {}
# for processo in df_amostra.index:
#     dict_results[processo] = {}
    
#     dados_de_capa = df_amostra.loc[processo,'dados_capa_agente']
#     prompt = make_prompt(processo, dados_de_capa, json_schema,usuario)
    
#     response: ChatResponse = chat(model='deepseek-r1:1.5b', 
#                               messages = [{'role': 'user','content':prompt}],
#                               format = ClassificacaoProcessos.model_json_schema(),
#                               options = {'temperature': 0}     
#                               )
#     try:
#         structured_response = ClassificacaoProcessos.model_validate_json(response.message.content)
#         flag = structured_response.flag
#         motivo = structured_response.motivo
#     except:
#         flag = "erro"
#         motivo = "erro"

#     # Variáveis
#     tokens_input = response.prompt_eval_count
#     tokens_output = response.eval_count
#     tempo_segundos = response.total_duration*1e-9
#     try:
#         tokens_total = tokens_input + tokens_output
#         tokens_por_segundo = (tokens_input+tokens_output) / tempo_segundos
#     except:
#         tokens_total = None
#         tokens_por_segundo = None


#     # populando o dict
#     resultado = {
#         'dados_de_capa_agente' : dados_de_capa,
#         'raw': response.message.content,
#         'flag': flag,
#         'motivo': motivo,
#         'tokens_input': tokens_input,
#         'tokens_output': tokens_output,
#         'tokens_total': tokens_total,
#         'tempo_segundos': tempo_segundos,
#         'tokens/segundo': tokens_por_segundo
#     }
#     dict_results[processo] = resultado

In [395]:
dict_results = {}
erro_output_estruturado = 'erro output estruturado'
for processo in df_amostra.index:
    dict_results[processo] = {}
    
    dados_de_capa = df_amostra.loc[processo,'dados_capa_agente']
    prompt = make_prompt(processo, dados_de_capa, json_schema,usuario)
    
    response: ChatResponse = chat(model='deepseek-r1:1.5b', 
                              messages = [{'role': 'user','content':prompt}],
                              options = {'temperature': 0}     
                              )
    
    # output_estruturado
    # regex = r'(\{\s*"flag":\s*".*?",\s*"motivo":\s*".*?"\s*\})'
    regex = r'\{\s*"flag"\s*:\s*".*?",\s*"motivo"\s*:\s*".*?"\s*\}'

    match = re.search(regex, response.message.content)
    try:
        if match:
            output_estruturado = match.group()
            output_estruturado = json.loads(output_estruturado)

        flag = output_estruturado.get('flag', erro_output_estruturado)
        motivo = output_estruturado.get('motivo', erro_output_estruturado)
    except:
        flag = erro_output_estruturado
        motivo = erro_output_estruturado

    # Variáveis
    tokens_input = response.prompt_eval_count
    tokens_output = response.eval_count
    tempo_segundos = response.total_duration*1e-9

    try:
        tokens_total = tokens_input + tokens_output
        tokens_por_segundo = (tokens_input+tokens_output) / tempo_segundos
    except:
        tokens_total = None
        tokens_por_segundo = None


    # populando o dict
    resultado = {
        'dados_de_capa_agente' : dados_de_capa,
        'raw': response.message.content,
        'flag': flag,
        'motivo': motivo,
        'tokens_input': tokens_input,
        'tokens_output': tokens_output,
        'tokens_total': tokens_total,
        'tempo_segundos': tempo_segundos,
        'tokens/segundo': tokens_por_segundo
    }
    dict_results[processo] = resultado

In [ ]:
# TO-DO: salvar para pickle
# TO-DO: salvar para pickle ao longo da execução

In [396]:
for processo in dict_results:
    print(f"processo: {processo}")
    resultado = dict_results[processo]
    for r in resultado.keys():
        print(f"{r}: {resultado[r]}")
    print('\n')
    print('#'*30)
    print('\n')


processo: 0000148-21.2024.5.05.0611
dados_de_capa_agente: <<Fonte: Tribunal Regional do Trabalho da 5ª Região
Classe da Ação: Ação Civil Pública Cível
Área do Direito: DIREITO PROCESSUAL CIVIL E DO TRABALHO
Assunto(s): DIREITO DO TRABALHO, Direito Coletivo do Trabalho, DIREITO PROCESSUAL CIVIL E DO TRABALHO,  Partes e Procuradores, Sucumbência , Direito Individual do Trabalho , Ação Civil Pública , Negociação Coletiva Trabalhista , Acordo e Convenção Coletivos de Trabalho , Honorários na Justiça do Trabalho , Impenhorabilidade , Multa Convencional , Contrato Individual de Trabalho, PIS/RAIS - Cadastramento, Verbas Remuneratórias, Indenizatórias e Benefícios, Cesta Básica, Plano de Saúde, Salário/Diferença Salarial, Salário Base - Obediência ao Salário Mínimo, Responsabilidade Solidária/Subsidiária, Liquidação / Cumprimento / Execução
>>

<<Fonte: Tribunal Regional do Trabalho da 5ª Região
Classe da Ação: Recurso Ordinário Trabalhista
Área do Direito: DIREITO PROCESSUAL CIVIL E DO TRABA